# Imports

In [12]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import re
import random
from tqdm import tqdm
from tqdm import tnrange
from tqdm import tqdm_notebook


# Model

# v1 - Current
* One layer of LSTM
* Non-batch train

In [36]:
with tf.Graph().as_default():
    # Graph
    # Dataset
    embed = hub.Module("https://tfhub.dev/google/nnlm-en-dim50/1")
    string2idtable = tf.contrib.lookup.index_table_from_file(vocabulary_file="vocabulary-shakespeare.txt", num_oov_buckets=0)
    id2stringtabel = tf.contrib.lookup.index_to_string_table_from_file(vocabulary_file="vocabulary-shakespeare.txt")
    def _insertSpace(sentence):
        sentence = sentence.decode()
        sentence = sentence.lower()
        sentence = re.sub(r'([\W\d])', r' \1 ', sentence)
        return sentence
    # Use tf.string_split if want to split string
    # def _split(sentence):
    #    return sentence.split()
    def _getLabel(sentence):
        splited = tf.string_split([sentence]).values
        sentence = splited
        # Use " " as x_0
        # sentence = tf.concat([tf.constant([" "], dtype=tf.string), sentence[0:-1]], 0)
        sentence = sentence[0:-1]
        ids = string2idtable.lookup(splited)
        ids = tf.one_hot(ids, 11405)
        return sentence, ids# {"sentence":sentence}, ids
    filenames = ["poems/shakespeare/sonnets.txt"]
    # TODO Use skip and filter methods to preprocess data rather than manually do it
    # TODO Use Dataset.map method to map '\n' to 'xxxnewlinexxx'
    dataset = tf.data.TextLineDataset(filenames)
    dataset = dataset.map(lambda sentence: tf.py_func(_insertSpace, [sentence], tf.string))
    dataset = dataset.map(_getLabel)
    # dataset = dataset.shuffle(buffer_size=10000)
    # dataset = dataset.batch(4)
    dataset = dataset.repeat()
    iterator = dataset.make_initializable_iterator()
    next_poem = iterator.get_next()

    # Variables
    """
    def get_reuse_var(name, shape, scope="dense", initializer=tf.random_normal_initializer):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            var = tf.get_variable(name, shape=shape, initializer=initializer)
        return var
    softmax_w = get_reuse_var("softmax_w", shape=[400, 11405])
    softmax_b = get_reuse_var("softmax_b", shape=[1, 11405])
    """
    softmax_w = tf.get_variable("softmax_w", shape=[400, 11405], initializer=tf.random_normal_initializer)
    softmax_b = tf.get_variable("softmax_b", shape=[1, 11405], initializer=tf.random_normal_initializer)

    # TODO Use tuple for state
    lstm = tf.contrib.rnn.LSTMCell(400, state_is_tuple=False, initializer=tf.random_normal_initializer, reuse=tf.AUTO_REUSE, name="LSTM1")

    # Model
    sentence, label = next_poem
    sentence = embed(sentence)
    # sentence = tf.concat([tf.zeros([1, 50]), sentence[1:]], 0)
    sentence = tf.concat([tf.zeros([1, 50]), sentence], 0, name="Insert_X_0")


    state = tf.placeholder(shape=[1, lstm.state_size], dtype=tf.float32, name="Previous_State")
    # TODO All loop must move to session??
    # for _ in range(total_steps):
    #x, y = sentence, label

    x = tf.placeholder(shape=[50], dtype=tf.float32, name="Input_Word")
    y = tf.placeholder(shape=[11405], dtype=tf.int32, name="Target_Word")
    # TODO How to Access timestep?
    input_word = tf.reshape(x, [1, 50])
    output, out_state = lstm(input_word, state)
    logits = tf.add(tf.matmul(output, softmax_w), softmax_b)
    possibility = tf.nn.softmax(logits=logits, name="Possibilities")
    word_index = tf.argmax(possibility, axis=-1, name="Predict_Word_index")
    word = id2stringtabel.lookup(word_index)
    loss_op = tf.losses.softmax_cross_entropy(onehot_labels=[y], logits=logits)
    tf.summary.scalar(name="loss", tensor=loss_op)
    merged = tf.summary.merge_all()
    # loss_op = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=logits, name="Loss")

    #total_loss = tf.get_variable(shape=[], dtype=tf.float32, name="Total_Cost", initializer=tf.zeros_initializer)

    # TODO Train Ops
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss_op)
    
    # sess = tf.Session()
    with tf.Session() as sess:
        writer = tf.summary.FileWriter("tmp/log/", sess.graph)
        sess.run(tf.global_variables_initializer())
        tf.tables_initializer().run(session=sess)
        sess.run(iterator.initializer)
        total_steps = 1000
        # total_loss = 0.0
        # TODO All loop must move to session??
        global_step = 1
        for _ in range(total_steps):
            wordsVec, targets = sess.run([sentence, label])
            # TODO How to Access timestep?
            # Initial
            _, pre_state = sess.run([output, out_state], feed_dict={x: wordsVec[0], y: targets[0], state: np.zeros([1, lstm.state_size], dtype=float)})
            for i in range(1, wordsVec.shape[0]):
                # total_loss += loss_op
                _, pre_state, loss, summary, out = sess.run([train_op, out_state, loss_op, merged, word], feed_dict={x: wordsVec[i], y: targets[i], state: pre_state})
                writer.add_summary(summary, global_step=global_step)
                tf.logging.log_every_n(tf.logging.INFO, "Loss: %s | local step: %s | global step: %s | Output: %s", 100, loss, i, global_step, out)
                global_step += 1
        writer.close()

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/var/folders/qz/dx3zfgtj2lqf70kv8b90s8wr0000gp/T/tfhub_modules/7f07056e3a4c9f125d5bd920ef3883605d8556a8/variables/variables' with embeddings
INFO:tensorflow:Loss: 17.51368 | local step: 1 | global step: 1 | Output: [b'ft']
INFO:tensorflow:Loss: 37.096455 | local step: 101 | global step: 101 | Output: [b'authorized']
INFO:tensorflow:Loss: 34.36387 | local step: 51 | global step: 201 | Output: [b'enhancing']
INFO:tensorflow:Loss: 42.92499 | local step: 151 | global step: 301 | Output: [b'esteeming']
INFO:tensorflow:Loss: 42.02422 | local step: 90 | global step: 401 | Output: [b'prepared']
INFO:tensorflow:Loss: 28.092344 | local step: 41 | global step: 501 | Output: [b'friendship']
INFO:tensorflow:Loss: 38.09091 | local step: 141 | global step: 601 | Output: [b'desk']
INFO:tensorflow:Loss: 27.584349 | local step: 100 | global step: 701 | Output: [b'positive']
INFO:tensorflow:Loss: 52.29058 | local step: 53 | 

KeyboardInterrupt: 

## Predict

In [37]:

idice = id2stringtabel.lookup(tf.constant([random.randint(0, 11404)], dtype=tf.int64))
idix, prev_state = sess.run([idice, out_state], feed_dict={x: np.zeros([50]), state: np.zeros([1, lstm.state_size], dtype=float)})
idix = str(idix[0])
print(idix)
w = sess.run(tf.reshape(embed([idix]), shape=[50]))
idice = id2stringtabel.lookup(tf.argmax(word))
for i in range(10):
    idix, prev_state = sess.run([idice, out_state], feed_dict={x:w, state: prev_state})
    w = str(idix)
    print(w)
    w = sess.run(tf.reshape(embed([w]), shape=[50]))
                                        

ValueError: Tensor("Const_54:0", shape=(1,), dtype=int64) must be from the same graph as Tensor("index_to_string:0", shape=(), dtype=resource).

# Bidirectional and Multi-RNN